In [38]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model


In [39]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download the stopwords list (only once)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

##Non-Open ended question

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util


# Define keywords or attribute phrases

# Example description



# Load a pre-trained sentence transformer model

# Function to score a description
def club_score(description, attributes):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    scores = {}
    desc_embedding = model.encode(description, convert_to_tensor=True)

    for attr, phrases in attributes.items():
        attr_embeddings = model.encode(phrases, convert_to_tensor=True)
        similarity = util.cos_sim(desc_embedding, attr_embeddings)
        scores[attr] = similarity.mean().item()  # Average similarity score

    return scores



# Assess the description
def match_clubs(club_descriptions,user_attributes):
  matching_clubs = {}
  filtered_matching_clubs = {}
  attributes = { #WARNING: Words are subjected to change
    "teamwork": ["joint work","networking", "collaboration", "cooperative", "partnership", "coordinate","support", "collective effort","mutual", "team effort"], # "cooperation", "team", "collaboration", "group work", "mutual support", "collective effort", "synergy", "shared responsibility", "coordination", "joint effort", "supportive environment", "team spirit", "interdependence", "togetherness", "cooperation across teams", "team-based", "group dynamics"
    "community_service": ["awareness","community", "volunteer", "outreach", "giving back", "charity","society","non-profit","social impact","public benefit"], # "charity", "social impact", "community engagement","helping others", "civic responsibility", "social responsibility", "philanthropy", "mission trips", "voluntary service", "empowerment", "public service", "nonprofit work", "community involvement", "advocacy"
    "leadership": ["inspire", "initiative", "motivated", "achieving","leader", "responsible","excel", "succeed", "evolve","future",], # "leadership", "leader", "inspire", "mentor", "guide", "decision-making", "visionary","responsibility", "motivator", "manager", "influence", "coach", "authority", "empower", "director", "organizer", "strategic thinking", "leadership development", "leading by example"
    "eagerness_to_learn": ["interest","passion","curious","challenge", "learn", "grow", "knowledge","explore","develop", "innovation"], #"learning", "curiosity", "knowledge-seeking", "growth mindset", "self-improvement", "development", "education", "exploration", "adaptability", "open-mindedness", "lifelong learning", "innovation","self-driven", "intellectual curiosity", "new experiences", "skill-building", "challenging oneself","personal growth", "exploration of new ideas", "knowledge expansion"
    "critical_thinking": ["analysis", "problem-solving", "decision-making", "evaluate", "strategy", "analytical","critical thinking","reasoning","logic","insight"] # "problem-solving", "analysis", "critical thinking", "reasoning", "evaluation", "decision-making", "strategy","logic", "objectivity", "judgment", "reflection", "inquiry", "debate", "data analysis", "questioning assumptions","cognitive skills", "thoughtful consideration", "synthesis", "problem resolution", "insight"
}
  #remove this part later
  for i in attributes:
    print(len(attributes[i]))

  for i in range(len(club_descriptions)):
    categories = ["teamwork", "community_service", "leadership", "eagerness_to_learn", "critical_thinking"]
    scores = dict(sorted(club_score(club_descriptions[i], attributes).items(), key=lambda item: item[1]))
    print(scores) #remove this later
    for attr in range(len(user_attributes)):
      if (user_attributes[attr]) and scores[categories[attr]] > 0.1:
        matching_clubs[i] = scores[categories[attr]]

  if len(matching_clubs) <= 4:
    for j in matching_clubs:
      if len(filtered_matching_clubs) == 2:
        break
      if matching_clubs[j] > 0.1:
        filtered_matching_clubs[j] = matching_clubs[j]
  elif len(matching_clubs) <= 8:
    for j in matching_clubs:
      if len(filtered_matching_clubs) == 3:
        break
      if matching_clubs[j] > 0.12:
        filtered_matching_clubs[j] = matching_clubs[j]
  elif len(matching_clubs) > 8:
    for j in matching_clubs:
      if len(filtered_matching_clubs) == 5:
        break
      if matching_clubs[j] > 0.15:
        filtered_matching_clubs[j] = matching_clubs[j]

  sorted_dict = dict(sorted(filtered_matching_clubs.items(), key=lambda item: item[1]))
  return list(sorted_dict.keys())[::-1]



##Open_Ended Question

In [57]:
import re
import numpy as np



#filtering sentence to make similarity scores more relevent
def filter_sentence(sentence):

  # Step 1: Tokenize the sentence into words
  words = word_tokenize(sentence)

  # Step 2: Get the list of stopwords in English
  stop_words = set(stopwords.words('english'))

  # Step 3: Remove stopwords from the sentence
  filtered_words = [word for word in words if word.lower() not in stop_words]

  # Step 4: Reconstruct the sentence without stopwords
  filtered_sentence = ' '.join(filtered_words)

  # Output the result
  return filtered_sentence

#finding relevent clubs
#TODO: Might need to split a group of sentences into individual sentences and check mean of similarity score for each sentence
def interest_similarity(club_descriptions, user_input):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  unfiltered_clubs = {}
  filtered_clubs = {}
  for i in range(len(club_descriptions)):
    mean = []
    sentences = re.split(r'[.!?]', club_descriptions[i])
    sentences = [s.strip() for s in sentences if s.strip()]
    for a in range(len(sentences)):
      sentences[a] = filter_sentence(sentences[a])

      user_embedding = model.encode(user_input, convert_to_tensor=True)
      club_embedding = model.encode(sentences[a], convert_to_tensor=True)
      cosine_similarity = util.cos_sim(user_embedding, club_embedding)
      mean.append(cosine_similarity)
    unfiltered_clubs[i] = np.mean(np.array(mean))

  if len(unfiltered_clubs) <= 4:
    for j in unfiltered_clubs:
      if len(filtered_clubs) == 2:
        break
      if unfiltered_clubs[j] > 0.12:
        filtered_clubs[j] = unfiltered_clubs[j]
  elif len(unfiltered_clubs) <= 8:
    for j in unfiltered_clubs:
      if len(filtered_clubs) == 3:
        break
      if unfiltered_clubs[j] > 0.15:
        filtered_clubs[j] = unfiltered_clubs[j]
  elif len(unfiltered_clubs) > 8:
    for j in unfiltered_clubs:
      if len(filtered_clubs) == 5:
        break
      if unfiltered_clubs[j] > 0.2:
        filtered_clubs[j] = unfiltered_clubs[j]

  sorted_dict = dict(sorted(filtered_clubs.items(), key=lambda item: item[1]))
  return list(sorted_dict.keys())[::-1]







#Final Implementation

In [58]:

def indicesOfMatchingClubs(club_descriptions, teamwork,community_service,leadership,learning,critical_thinking,hobbies,club_wants):
  questions = {"teamwork": teamwork,
          "community_service": community_service,
          "leadership": leadership,
          "learning": learning,
          "critical_thinking": critical_thinking,
          "hobbies": hobbies,
          "club_wants": club_wants
          }
  user_input = [questions["hobbies"] + ". " + questions["club_wants"]]
  user_attributes = [questions["teamwork"],questions["community_service"],questions["leadership"],questions["learning"],questions["critical_thinking"]]

  return list(np.unique(np.array((interest_similarity(club_descriptions, user_input) + match_clubs(club_descriptions, user_attributes)))))

def listOfMatchingClubs(event_descriptions,club_titles,club_descriptions, teamwork,community_service,leadership,learning,critical_thinking,hobbies,club_wants):
  finalClubs = {}
  for i in indicesOfMatchingClubs(club_descriptions, teamwork,community_service,leadership,learning,critical_thinking,hobbies,club_wants):
    finalClubs[club_titles[i]] = event_descriptions[i]
  return finalClubs


#Testing the model

In [43]:
pip install flask-cors

In [44]:
from flask import Flask, request, jsonify
import firebase_admin
from firebase_admin import credentials, db
from flask_cors import CORS
import csv
# Initialize Flask app
app = Flask(__name__)

# Firebase Admin SDK initialization
# cred = credentials.Certificate("path/to/your/firebase-service-account.json")
# firebase_admin.initialize_app(cred, {
#     'databaseURL': 'https://your-database-name.firebaseio.com/'
# })
CORS(app)

# Read the organizations data CSV file
with open('organizations_data.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    organizations_data = [row for row in csv_reader]


# Extract only the descriptions from the organizations data
club_descriptions = [row['description'] for row in organizations_data]
club_title = [row['name'] for row in organizations_data]
# Read the event details CSV file
with open('event_details.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    event_details = [row for row in csv_reader]

# Print the event details to verify
event_descriptions = [row['description'] for row in event_details]


# Print the event details to verify
print(event_details)
# Dummy AI model function
def dummy_ai_model(user_data):
    """
    Placeholder function for AI model processing.
    Takes user data as input and returns a dummy result.
    """
    dummy_result = listOfMatchingClubs(event_descriptions,club_title,club_descriptions,user_data.teamwork,user_data.community_service,user_data.leadership,user_data.learning,user_data.critical_thinking,user_data.hobbies,user_data.club_wants)
    # Replace this with your actual AI model logic
    return {"prediction": "success", "confidence": 0.95}

@app.route('/submit', methods=['POST'])
def submit_user_data():
    try:
        # Get JSON data from the request
        user_data = request.json
        print(user_data)
        # # Send schema to Firebase
        # ref = db.reference('users')  # Creates a 'schemas' node in the Firebase DB
        # ref.push(schema)  # Pushes the schema to the database

         # Run the AI model on the user data
        ai_result = dummy_ai_model(user_data)

        # Combine and return the results
        response = {
            "firebase_status": "User data sent successfully",
            "ai_result": ai_result
        }

        return jsonify(ai_result), 200

    except Exception as e:
        # Handle errors
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: 'organizations_data.csv'

In [61]:
questions = {"teamwork": False,
            "community_service": True,
            "leadership": False,
            "learning": True,
            "critical_thinking": True,
            "hobbies": "I love dancing, especially Bollywood-style dance.",
            "club_wants": "I want a club that can help me pursue a career in international relations."
            }
club_descriptions = [
    "Aa Dekhen Zara (ADZ) is an annual South Asian Dance competition hosted by the students of the University of Wisconsin-Madison. We host talented Bollywood-Fusion dance teams.",
    "The Alexander Hamilton Society (AHS) is a non-partisan organization focused on foreign policy and national security international relations.",
    "The American Medical College of Pharmacy (AMCP) provides opportunities to learn about managed care pharmacy, improving patient outcomes through clinical and business knowledge.",
    "Acacia is a social fraternity dedicated to human service. We are dedicated to bettering the community we are a part of and sharing a brotherhood we will remember for the rest of our lives. We are a fraternity that is focused on the strength of brotherhood. By selecting key members of the Madison community, we build a brotherhood dedicated to human service for both the community and ourselves. These members are selected through the IFC-regulated RUSH process in which members are first bid followed by a pledge process. We are a non-hazing fraternity built on the idea that disrespect to any possible future member or person is strictly against our goals and morals. All leaders of the Acacia Fraternity are elected by the active members and must be students of the University of Wisconsin-Madison.  ",
"The Health Entreprenuers and Leaders is a prehealth student organization established at the University of Wisconsin-Madison in 2020. As prehealth students, we identified the gaps and disparities in our community and the medical field. HEAL is our legacy to become the change that goes beyond addressing the issue but pushes forward to act upon them at the core of the issues. We understand Health care policy and insurance are crucial factors that comes to healthcare affordability and accessibility, which HEAL has a focus to address. Beyond this, the cornerstone of HEAL is to address problems in healthcare at all levels, including but not limited to the federal, state, hospital administration, healthcare providers/professionals, patients, etc. HEAL is structured to be a center for all pre-health students to attain resources and mentorship to grow and develop in their path to pursuing and entering the health field. HEAL understands the benefits and advocates for the diversity of people, backgrounds, and minds to improve the future of medicine."
]
club_titles = ["Aa Dekhen Zara", "The Alexander Hamilton Society", "The American Medical College of Pharmacy", "Acacia", "The Health Entreprenuers and Leaders"]
event_descriptions = ["event1", "event2", "event3","event4","event5"]
print(listOfMatchingClubs(event_descriptions, club_titles,club_descriptions,False,True,False,True,True,"I love dancing, especially Bollywood-style dance.","I want a club that can help me pursue a career in international relations."))

10
10
10
10
10
{'community_service': 0.10400845855474472, 'teamwork': 0.10795339196920395, 'eagerness_to_learn': 0.11105271428823471, 'leadership': 0.11280133575201035, 'critical_thinking': 0.11412223428487778}
{'eagerness_to_learn': 0.05439390987157822, 'leadership': 0.07518069446086884, 'teamwork': 0.09696871042251587, 'critical_thinking': 0.10065927356481552, 'community_service': 0.1673998236656189}
{'leadership': 0.027347946539521217, 'eagerness_to_learn': 0.04596151039004326, 'community_service': 0.05191246420145035, 'teamwork': 0.06298767030239105, 'critical_thinking': 0.06982514262199402}
{'eagerness_to_learn': 0.10840778052806854, 'critical_thinking': 0.11968077719211578, 'leadership': 0.1474871188402176, 'teamwork': 0.17062781751155853, 'community_service': 0.2042498141527176}
{'eagerness_to_learn': 0.11082278192043304, 'teamwork': 0.11181306838989258, 'critical_thinking': 0.12578324973583221, 'leadership': 0.13555608689785004, 'community_service': 0.21739566326141357}
{'Aa De